In [1]:
import pandas as pd
from PIL import Image
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
from tensorflow.keras import layers, Model
import random

In [2]:
trainPath = "/kaggle/input/trainingpatientmetadataforrsnabreastcancer/train.csv"

train_df = pd.read_csv(trainPath)
print(train_df.info())

train_data = train_df.drop(["cancer"], axis=1, inplace=False)
train_labels = train_df["cancer"]

for i in range(300):
    X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels,
        test_size=0.2, shuffle = False, random_state = 45)
X_val = X_val.reset_index()
y_val = y_val.reset_index()

probTrain = y_train.value_counts()[1]/(y_train.value_counts()[0]+y_train.value_counts()[1])
probVal = len(y_val[y_val.cancer==1])/(len(y_val[y_val.cancer==1])+len(y_val[y_val.cancer==0]))

print(probTrain, probVal)

# print(len(y_train[y_train.cancer==1]))
# print(len(y_val[y_val.cancer==1]))
# X_val.index = np.arange(0, len(X_val))
# y_val.index = np.arange(0, len(y_val))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   site_id                  54706 non-null  int64  
 1   patient_id               54706 non-null  int64  
 2   image_id                 54706 non-null  int64  
 3   laterality               54706 non-null  object 
 4   view                     54706 non-null  object 
 5   age                      54669 non-null  float64
 6   cancer                   54706 non-null  int64  
 7   biopsy                   54706 non-null  int64  
 8   invasive                 54706 non-null  int64  
 9   BIRADS                   26286 non-null  float64
 10  implant                  54706 non-null  int64  
 11  density                  29470 non-null  object 
 12  machine_id               54706 non-null  int64  
 13  difficult_negative_case  54706 non-null  bool   
dtypes: bool(1), float64(2)

In [3]:
imgBasePath = "/kaggle/input/rsna-breast-cancer-256-pngs/"
LCC_train = []
LMLO_train = []
RCC_train = []
RMLO_train = []

In [4]:
zero = 0
one = 0
for i in range(len(X_train)):
#     if(i == 100):
#       break
    rnd = random.random()
    laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
    label = y_train[i]
    pat_id = X_train._get_value(i, "patient_id")
    img_id = X_train._get_value(i, "image_id")
    imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
    #print(i, imgPath)
    #file = Path(imgPath)
    #img = Image.open(imgPath)
    #img = img.resize((256, 256))
    #imgArr = np.asarray(img)
    #imgArr = imgArr / 255
    
    if(laterality == "LCC"):
        if(label == 1 or rnd <= probTrain):
            LCC_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    elif(laterality == "LMLO"):
        if(label == 1 or rnd <= probTrain):
            LMLO_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    elif(laterality == "RCC"):
        if(label == 1 or rnd <= probTrain):
            RCC_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    elif(laterality == "RMLO"):
        if(label == 1 or rnd <= probTrain):
            RMLO_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    #print(len(X_train)-i, end=" ")

In [5]:
print(one/(zero+one))

0.5055741360089187


In [6]:
imgBasePath = "/kaggle/input/rsna-breast-cancer-256-pngs/"
LCC_val = []
LMLO_val = []
RCC_val = []
RMLO_val = []

In [7]:
print(X_val.head())

   index  site_id  patient_id    image_id laterality view   age  biopsy  \
0  43764        2       57164  2063547844          L  MLO  50.0       1   
1  43765        2       57164  1522121544          R   CC  50.0       0   
2  43766        2       57164  2060537563          R  MLO  50.0       0   
3  43767        1        5717   544564406          L   CC  59.0       0   
4  43768        1        5717  1031134031          L  MLO  59.0       0   

   invasive  BIRADS  implant density  machine_id  difficult_negative_case  
0         0     NaN        0     NaN          48                     True  
1         0     NaN        0     NaN          48                    False  
2         0     NaN        0     NaN          48                    False  
3         0     1.0        0       B          93                    False  
4         0     1.0        0       B          93                    False  


In [10]:
for i in range(len(X_val)):
#     if(i == 100):
#       break
    label = int(y_val.at[i, "cancer"])
    rnd = random.random()
    laterality = X_val._get_value(i, "laterality") + X_val._get_value(i, "view")
    # label = y_train[i]
    pat_id = X_val._get_value(i, "patient_id")
    img_id = X_val._get_value(i, "image_id")
    imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
    #print(i, imgPath)
    #file = Path(imgPath)
    #img = Image.open(imgPath)
    #img = img.resize((256, 256))
    #imgArr = np.asarray(img)
    #imgArr = imgArr / 255
    if(laterality == "LCC"):
        #if(label == 1 or rnd <= probVal):
        LCC_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    elif(laterality == "LMLO"):
        #if(label == 1 or rnd <= probVal):
        LMLO_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    elif(laterality == "RCC"):
        #if(label == 1 or rnd <= probVal):
        RCC_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    elif(laterality == "RMLO"):
        #if(label == 1 or rnd <= probVal):
        RMLO_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    #print(len(X_val)-i, end=" ")

In [11]:
print(one/(zero+one))

0.09078771695594126


In [12]:
print(len(X_val))
print(np.count_nonzero(RMLO_val==0))

10942
0


In [13]:
import random
def generator(filenames, X_dataset, y_dataset):
    def callable_generator():
        for filename in filenames:
            f = filename
            rf = f[len(imgBasePath):-4].split("_")
            pat_id = rf[0]
            img_id = rf[1]
            arr = X_dataset["image_id"].to_numpy()
            i = X_dataset.index[X_dataset['image_id'] == int(img_id)].tolist()
            label = float(y_dataset._get_value(i[0], "cancer"))
#             label = y_dataset.iloc(i)["cancer"]
#             print(label)
#             #print(type(label))
            img = Image.open(f)
            imgArr = np.asarray(img)
            imgArr = imgArr / 255
            imgArr = np.reshape(imgArr, (256, 256))
            yield imgArr, label
    return callable_generator

In [14]:
LCC_train_dataset = tf.data.Dataset.from_generator(generator(LCC_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
LMLO_train_dataset = tf.data.Dataset.from_generator(generator(LMLO_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RCC_train_dataset = tf.data.Dataset.from_generator(generator(RCC_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RMLO_train_dataset = tf.data.Dataset.from_generator(generator(RMLO_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))


In [15]:
LCC_val_dataset = tf.data.Dataset.from_generator(generator(LCC_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
LMLO_val_dataset = tf.data.Dataset.from_generator(generator(LMLO_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RCC_val_dataset = tf.data.Dataset.from_generator(generator(RCC_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RMLO_val_dataset = tf.data.Dataset.from_generator(generator(RMLO_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))

In [16]:
BATCH_SIZE = 16
def prep_data(train_dataset):
    train_dataset = train_dataset.cache().batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return train_dataset

In [17]:
LCC_train_dataset = prep_data(LCC_train_dataset)
LMLO_train_dataset = prep_data(LMLO_train_dataset)
RCC_train_dataset = prep_data(RCC_train_dataset)
RMLO_train_dataset = prep_data(RMLO_train_dataset)

In [18]:
# for lcc_batch in LCC_train_dataset:
#    print(lcc_batch[0])
# print("\n-------------------------------------------------------\n")

# for lcc_batch in LCC_val_dataset:
#    print(lcc_batch[0])

In [19]:
LCC_val_dataset = prep_data(LCC_val_dataset)
LMLO_val_dataset = prep_data(LMLO_val_dataset)
RCC_val_dataset = prep_data(RCC_val_dataset)
RMLO_val_dataset = prep_data(RMLO_val_dataset)

In [20]:
def convBlock(num_filters=[128,64],kernel_size=[7,5],stride=2):
    def customLayer(inputs):
        x = layers.Conv2D(num_filters[0],kernel_size[0],activation='relu')(inputs)
        x = layers.Conv2D(num_filters[1],kernel_size[1],activation='relu')(x)
        x = layers.MaxPooling2D(stride)(x)
        x = layers.BatchNormalization()(x)
        return x
    return customLayer

In [21]:
def cnnModel(input_shape,num_classes):
    inputs = layers.Input(input_shape)
    x = convBlock(num_filters=[128,64],kernel_size=[7,5],stride=2)(inputs)
    x = layers.Dropout(0.4)(x)
    x = convBlock(num_filters=[64,32],kernel_size=[5,3],stride=2)(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512,activation='relu')(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(num_classes,activation='softmax')(x)
    return Model(inputs=[inputs],outputs=[output])

In [22]:
EPOCHS=25
early_stopping=tf.keras.callbacks.EarlyStopping(patience=7,restore_best_weights=True)

In [23]:
# rm -rf /kaggle/working/

In [31]:
def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [34]:
model = cnnModel((256, 256, 1),num_classes=2)
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=["accuracy"])
LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(LCC_model.history) 
hist_df.to_csv("LCC_model_V1_hist.csv")
model.save("LCC_model_V1.h5")

Epoch 1/25


OperatorNotAllowedInGraphError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_25/2062793374.py", line 7, in pfbeta  *
        prediction = min(max(predictions[idx], 0), 1)

    OperatorNotAllowedInGraphError: Using a symbolic `tf.Tensor` as a Python `bool` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.


In [29]:
model = cnnModel((256, 256, 1),num_classes=2)
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=[pfbeta])
LMLO_model = model.fit(LMLO_train_dataset, epochs=EPOCHS, validation_data=LMLO_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(LMLO_model.history) 
hist_df.to_csv("LMLO_model_V1_hist.csv")
model.save("LMLO_model_V1.h5")

Epoch 1/25


TypeError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)

    TypeError: tf__pfbeta() missing 1 required positional argument: 'beta'


In [26]:
model = cnnModel((256, 256, 1),num_classes=2)
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
RCC_model = model.fit(RCC_train_dataset, epochs=EPOCHS, validation_data=RCC_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(RCC_model.history) 
hist_df.to_csv("RCC_model_V1_hist.csv")
model.save("RCC_model_V1.h5")

Epoch 1/25


2023-03-23 00:53:04.702638: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


29/29 [==============================] - 36s 1s/step - loss: 21.7400 - accuracy: 0.4758 - val_loss: 5.0851 - val_accuracy: 0.8071
Epoch 2/25
29/29 [==============================] - 16s 575ms/step - loss: 10.8502 - accuracy: 0.4714 - val_loss: 3.9678 - val_accuracy: 0.5334
Epoch 3/25
29/29 [==============================] - 16s 563ms/step - loss: 4.0120 - accuracy: 0.4912 - val_loss: 4.3625 - val_accuracy: 0.8554
Epoch 4/25
29/29 [==============================] - 16s 560ms/step - loss: 2.1736 - accuracy: 0.5110 - val_loss: 4.2245 - val_accuracy: 0.8189
Epoch 5/25
29/29 [==============================] - 17s 582ms/step - loss: 1.1170 - accuracy: 0.5286 - val_loss: 0.5542 - val_accuracy: 0.6167
Epoch 6/25
29/29 [==============================] - 16s 561ms/step - loss: 0.9308 - accuracy: 0.5396 - val_loss: 0.6968 - val_accuracy: 0.5758
Epoch 7/25
29/29 [==============================] - 16s 562ms/step - loss: 0.7648 - accuracy: 0.5595 - val_loss: 0.7925 - val_accuracy: 0.7894
Epoch 8/25


In [27]:
model = cnnModel((256, 256, 1),num_classes=2)
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
RMLO_model = model.fit(RMLO_train_dataset, epochs=EPOCHS, validation_data=RMLO_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(RMLO_model.history) 
hist_df.to_csv("RMLO_model_V1_hist.csv")
model.save("RMLO_model_V1.h5")

Epoch 1/25


2023-03-23 00:56:47.491527: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/dropout_9/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


28/28 [==============================] - 38s 1s/step - loss: 21.1084 - accuracy: 0.4743 - val_loss: 40.5550 - val_accuracy: 0.1987
Epoch 2/25
28/28 [==============================] - 16s 590ms/step - loss: 5.5414 - accuracy: 0.5235 - val_loss: 5.1731 - val_accuracy: 0.6690
Epoch 3/25
28/28 [==============================] - 16s 591ms/step - loss: 3.3025 - accuracy: 0.5481 - val_loss: 5.0619 - val_accuracy: 0.7674
Epoch 4/25
28/28 [==============================] - 16s 599ms/step - loss: 1.1368 - accuracy: 0.5391 - val_loss: 3.5186 - val_accuracy: 0.7652
Epoch 5/25
28/28 [==============================] - 16s 592ms/step - loss: 1.1507 - accuracy: 0.5213 - val_loss: 2.2977 - val_accuracy: 0.4162
Epoch 6/25
28/28 [==============================] - 16s 593ms/step - loss: 0.7243 - accuracy: 0.5145 - val_loss: 1.7465 - val_accuracy: 0.0700
Epoch 7/25
28/28 [==============================] - 16s 593ms/step - loss: 0.6546 - accuracy: 0.5705 - val_loss: 1.2427 - val_accuracy: 0.7184
Epoch 8/25


In [28]:
# # LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset, callbacks=[early_stopping], steps_per_epoch=250, validation_steps=250)
# LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset)
# hist_df = pd.DataFrame(LCC_model.history) 
# hist_df.to_csv("LCC_model_V1_hist.csv")
# model.save("LCC_model_V1.h5")

In [29]:
# LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset)

In [30]:
# rm -rf /kaggle/working/

In [31]:
# for i in range(len(X_train)):
#     if(i == 100):
#       break
    
#     laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#     label = y_train[i]
#     pat_id = X_train._get_value(i, "patient_id")
#     img_id = X_train._get_value(i, "image_id")
#     imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#     #print(i, imgPath)
#     #file = Path(imgPath)
#     img = Image.open(imgPath)
#     #img = img.resize((256, 256))
#     imgArr = np.asarray(img)
#     imgArr = imgArr / 255
#     if(laterality == "LCC"):
#         LCC_X_train.append(imgArr)
#         LCC_y_train.append(label)
#     elif(laterality == "LMLO"):
#         LMLO_X_train.append(imgArr)
#         LMLO_y_train.append(label)
#     elif(laterality == "RCC"):
#         RCC_X_train.append(imgArr)
#         RCC_y_train.append(label)
#     elif(laterality == "RMLO"):
#         RMLO_X_train.append(imgArr)
#         RMLO_y_train.append(label)
#     print(len(X_train)-i, end=" ")

In [32]:
# imgBasePath = "/kaggle/input/rsna-breast-cancer-256-pngs/"
# LCC_X_train = []
# LCC_y_train = []

# LMLO_X_train = []
# LMLO_y_train = []

# RCC_X_train = []
# RCC_y_train = []

# RMLO_X_train = []
# RMLO_y_train = []

In [33]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
    
#     laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#     label = y_train[i]
#     pat_id = X_train._get_value(i, "patient_id")
#     img_id = X_train._get_value(i, "image_id")
#     imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#     #print(i, imgPath)
#     #file = Path(imgPath)
#     img = Image.open(imgPath)
#     #img = img.resize((256, 256))
#     imgArr = np.asarray(img)
#     imgArr = imgArr / 255
#     if(laterality == "LCC"):
#         LCC_X_train.append(imgArr)
#         LCC_y_train.append(label)
#     elif(laterality == "LMLO"):
#         LMLO_X_train.append(imgArr)
#         LMLO_y_train.append(label)
#     elif(laterality == "RCC"):
#         RCC_X_train.append(imgArr)
#         RCC_y_train.append(label)
#     elif(laterality == "RMLO"):
#         RMLO_X_train.append(imgArr)
#         RMLO_y_train.append(label)
#     print(len(X_train)-i, end=" ")

In [34]:
# LCC_X_train = np.array(LCC_X_train)
# LCC_y_train = np.array(LCC_y_train)

# # LMLO_X_train = np.array(LMLO_X_train)
# # LMLO_y_train = np.array(LMLO_y_train)

# # RCC_X_train = np.array(RCC_X_train)
# # RCC_y_train = np.array(RCC_y_train)

# # RMLO_X_train = np.array(RMLO_X_train)
# # RMLO_y_train = np.array(RMLO_y_train)

# print(LCC_X_train.shape, LCC_y_train.shape)
# # print(LMLO_X_train.shape, LMLO_y_train.shape)
# # print(RCC_X_train.shape, RCC_y_train.shape)
# # print(RMLO_X_train.shape, RMLO_y_train.shape)

In [35]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
#     if(not (i in done)):
#         laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#         label = y_train[i]
#         pat_id = X_train._get_value(i, "patient_id")
#         img_id = X_train._get_value(i, "image_id")
#         imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#         #print(i, imgPath)
#         #file = Path(imgPath)
#         img = cv2.imread(imgPath, 0)
#         #img = img.resize((224))
#         imgArr = np.asarray(img, dtype=np.uint16)
#         imgArr = imgArr / 255
#         imgArr = np.reshape(imgArr, (1, 256, 256))
#     #     print(imgArr.nbytes)
#     #     print(sparse.COO(imgArr).nbytes)
#         # print(imgArr.shape)
#         if(laterality == "LMLO"):
#             LMLO_X_train = np.append(LMLO_X_train, imgArr, axis=0)
#             LMLO_y_train = np.append(LMLO_y_train, label)
#             done.append(i)
#     print(len(X_train)-i)

In [36]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
#     if(not (i in done)):
#         laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#         label = y_train[i]
#         pat_id = X_train._get_value(i, "patient_id")
#         img_id = X_train._get_value(i, "image_id")
#         imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#         #print(i, imgPath)
#         #file = Path(imgPath)
#         img = cv2.imread(imgPath, 0)
#         #img = img.resize((224))
#         imgArr = np.asarray(img, dtype=np.uint16)
#         imgArr = imgArr / 255
#         imgArr = np.reshape(imgArr, (1, 256, 256))
#     #     print(imgArr.nbytes)
#     #     print(sparse.COO(imgArr).nbytes)
#         # print(imgArr.shape)
#         if(laterality == "RCC"):
#             RCC_X_train = np.append(RCC_X_train, imgArr, axis=0)
#             RCC_y_train = np.append(RCC_y_train, label)
#             done.append(i)
#     print(len(X_train)-i)

In [37]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
#     if(not (i in done)):
#         laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#         label = y_train[i]
#         pat_id = X_train._get_value(i, "patient_id")
#         img_id = X_train._get_value(i, "image_id")
#         imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#         #print(i, imgPath)
#         #file = Path(imgPath)
#         img = cv2.imread(imgPath, 0)
#         #img = img.resize((224))
#         imgArr = np.asarray(img, dtype=np.uint16)
#         imgArr = imgArr / 255
#         imgArr = np.reshape(imgArr, (1, 256, 256))
#     #     print(imgArr.nbytes)
#     #     print(sparse.COO(imgArr).nbytes)
#         # print(imgArr.shape)
#         if(laterality == "RMLO"):
#             RMLO_X_train = np.append(RMLO_X_train, imgArr, axis=0)
#             RMLO_y_train = np.append(RMLO_y_train, label)
#             done.append(i)
#     print(len(X_train)-i)

In [39]:
# rm -rf /kaggle/working/

In [40]:
# np.save("LCCXarr", LCC_X_train)
# np.save("LCCyarr", LCC_y_train)

# np.save("LMLOXarr", LMLO_X_train)
# np.save("LMLOyarr", LMLO_y_train)

# np.save("RCCXarr", RCC_X_train)
# np.save("RCCyarr", RCC_y_train)

# np.save("RMLOXarr", RMLO_X_train)
# np.save("RMLOyarr", RMLO_y_train)

In [41]:
# LMLO_X_train.shape
# print(LMLO_y_train)